In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup # https://realpython.com/beautiful-soup-web-scraper-python/
import time
from datetime import date

# GRAB ALL TICKERS INSIDE PRICE (MIN-MAX) RANGE

In [2]:
pages = []
tickers_list = []
tickers_group_list = []

ticker_skip = 1900
price_min = 5.00
price_max = 25.00
valid_page_status = True

start_time = time.time()

while (valid_page_status):    
    
    url =   f"""
            https://www.marketwatch.com/tools/screener/stock?exchange=all&skip={ticker_skip}
            &orderbyfield=&direction=desc
            &visiblecolumns=Symbol,CompanyName,Price,NetChange,ChangePercent,Volume
            &pricemin={price_min}&pricemax={price_max}
            """ # must be inside while loop because {page_skip} won't update if outside of it...

    page = requests.get(url)
    soup_obj = soup(page.content, 'html.parser')
    ticker_skip += 25
    
    cells_found = soup_obj.find(class_ = "table__cell j-Symbol")

    if(cells_found):
        pages.append(url)
    else:
        valid_page_status = False
    
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Length of Pages[]: {len(pages)}")    
print(f"Elapsed Time: {elapsed_time}")

Length of Pages[]: 6
Elapsed Time: 10.003091096878052


In [3]:
for item in pages:
    page = requests.get(item)

    df = pd.read_html(page.content)

    tickers = df[0]
    tickers = tickers["Symbol"]
    
    for t in tickers:
        tickers_list.append(t)
        
    tickers_group_list.append(tickers)
    

import csv    
today = date.today()
date_format = today.strftime("%b-%d-%Y")
file_name = "tickers_" + date_format

with open(file_name, 'w') as f:
     
    write = csv.writer(f)
    write.writerows(tickers_group_list) 

# GRAB PRICE CHART TABLES FOR EVERY TICKER

In [5]:
month = today.strftime("%B").lower()
year = today.strftime("%Y")
day = today.strftime("%d")
unix_time_yesterday = -1
unix_time_5years = -1

unix_converter_url = f"https://timeconverter.online/list/{year}/{month}"
page = requests.get(unix_converter_url)
soup_obj = soup(page.content, 'html.parser')
table_rows = soup_obj.find_all("td")

# Get the unix time stamp for today's date
for item in table_rows:
    date_td = item.text.split(" ")
    if(date_td[0] == day):
        unix_time_yesterday = item.find_previous().find_previous().text
        # find_previous() written twice, since having been read already, the iterator has moved
        
year = str((int(year) - 5))
unix_converter_url = f"https://timeconverter.online/list/{year}/{month}"
page = requests.get(unix_converter_url)
soup_obj = soup(page.content, 'html.parser')
table_rows = soup_obj.find_all("td") 

# Get the unix time stamp for date from 5 years ago
for item in table_rows:
    date_td = item.text.split(" ")
    if(date_td[0] == day):
        unix_time_5years = item.find_previous().find_previous().text
        
main_url = f"https://finance.yahoo.com/quote/{tickers_list[0]}?p={tickers_list[0]}"
        
base_url = f"https://finance.yahoo.com/quote/{tickers_list[0]}/history?p={tickers_list[0]}"

weekly_url =    f"""
                https://finance.yahoo.com/quote/{tickers_list[0]}/history?
                period1={unix_time_5years}&period2={unix_time_yesterday}
                &interval=1wk&filter=history&frequency=1wk
                &includeAdjustedClose=true    
                """
# Yahoo Finance blocking scraping here????
page = requests.get(main_url)

df = pd.read_html(page.content)

soup_obj = soup(page.content, 'html.parser')
table_rows = soup_obj.find_all("td")
print(table_rows)


[                                                   0
 0  Will be right back...  Thank you for your pati...]